# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
#----------------------------------------------------------------------------------------
# DEPENDANCIES
#----------------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

#----------------------------------------------------------------------------------------
# IMPORT API KEY
#----------------------------------------------------------------------------------------
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#----------------------------------------------------------------------------------------
# READ CITIES CSV FILE
#----------------------------------------------------------------------------------------
weather_data_path = "../WeatherPy/cities_df.csv"
weather_data = pd.read_csv(weather_data_path)

#----------------------------------------------------------------------------------------
# DROP UNNAMED COLUMN
#----------------------------------------------------------------------------------------
weather_data = weather_data[['City', 'Latitude', 'Longitude', 'Temperature',
       'Humidity', 'Cloudiness', 'Wind_Speed']]

#----------------------------------------------------------------------------------------
# PREVIEW DATAFRAME
#----------------------------------------------------------------------------------------
weather_data.head()

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind_Speed
0,Hartselle,34.4434,-86.9353,21.67,93,29,2.33
1,Cabiguan,12.9058,123.7314,29.81,69,100,2.65
2,Nang Rong,14.6377,102.7914,33.95,46,4,2.40
3,Pangnirtung,66.1451,-65.7125,-5.55,94,100,2.51
4,Lālpur,22.2000,69.9667,29.00,50,0,5.31


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#----------------------------------------------------------------------------------------
# STORE REQUIRED LAT/LONG AND HUMIDITY DATA IN VARIABLES
#----------------------------------------------------------------------------------------
locations = weather_data[["Latitude", "Longitude"]]
humidity = weather_data["Humidity"]

In [4]:
#----------------------------------------------------------------------------------------
# CREATE HEATMAP
#----------------------------------------------------------------------------------------
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)
                                 
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#----------------------------------------------------------------------------------------------------
# FIND CITIES WITH PERFECT CONDITIONS. DROP ANY ROWS THAT DON"T CONTAIN ALL THREE CONDITIONS BELOW
#--- TEMPERATURE BETWEEN 21 & 27 DEGREES CENTIGRADE
#--- WIND SPEED LESS THAN 10 MPH
#--- ZERO CLOUDINESS
#----------------------------------------------------------------------------------------------------

# Filter the dataframe
perfect_weather_df = weather_data[(weather_data['Temperature'] >= 21) & (weather_data['Temperature'] < 23) & (weather_data['Wind_Speed'] < 10) & (weather_data['Cloudiness'] == 0)]

# Visualize
print(len(perfect_weather_df))
perfect_weather_df.head()


9


,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind_Speed
42,Luxor,25.6989,32.6421,21.96,23,0,2.32
94,San Isidro,-12.1167,-77.0500,21.12,75,0,3.09
136,Bellview,30.4616,-87.3150,22.68,95,0,2.68
250,Arvin,35.2091,-118.8284,21.01,51,0,1.91
304,Badiraguato,25.3667,-107.5167,22.61,37,0,1.30


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#----------------------------------------------------------------------------------------
# STORE CITIES WITH DESIRED WEATHER INTO VARIABLE NAMED 'HOTEL_DF' AND ADD NEW COLUMN
#----------------------------------------------------------------------------------------
hotel_df = perfect_weather_df
hotel_df["Hotel Name"] = ''
hotel_df.head(20)



C:\Users\rjtur\AppData\Local\Temp/ipykernel_9028/2354168230.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ''


,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind_Speed,Hotel Name
42,Luxor,25.6989,32.6421,21.96,23,0,2.32,
94,San Isidro,-12.1167,-77.0500,21.12,75,0,3.09,
136,Bellview,30.4616,-87.3150,22.68,95,0,2.68,
250,Arvin,35.2091,-118.8284,21.01,51,0,1.91,
304,Badiraguato,25.3667,-107.5167,22.61,37,0,1.30,
323,Marshall,34.3834,-86.3333,21.84,71,0,1.54,
324,Ayia Anna,34.9399,33.4843,21.44,72,0,2.06,
338,Meadville,41.6414,-80.1515,22.57,62,0,2.57,
613,Jandaia do Sul,-23.6031,-51.6433,22.36,89,0,3.24,


In [13]:
# #----------------------
# # TESTING BLOCK
# #----------------------

# target_coordinates = "25.6989, 32.6421"
# target_search = "hotel"
# target_radius = 5000

# params = {
#     "location": target_coordinates,
#     "keyword": target_search,
#     "radius": target_radius,
#     "key": g_key
# }


# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# # run a request using our params dictionary
# response = requests.get(base_url, params=params)
# # convert response to json
# hotels_data = response.json()   

# print(hotels_data["results"][0]["name"])

Hilton Luxor Resort & Spa


In [7]:
#----------------------------------------------------------------------------------------
# SET PARAMETERS TO SEARCH FOR HOTELS WITHIN 5000 METERS
# HIT THE GOOGLE PLACES API FOR EACH CITY"S COORDINATES
#----------------------------------------------------------------------------------------
hotel_list = []

cities_data = hotel_df["City"]
# lat = hotel_df["Latitude"]
# long = hotel_df["Longitude"]

for city in cities_data:

    lat = hotel_df[hotel_df["City"] == city]["Latitude"].values[0]
    long = hotel_df[hotel_df["City"] == city]["Longitude"].values[0]

    # print (lat,long)

    # target_coordinates = lat, long
    target_coordinates = str(lat) + ", " + str(long)
    target_search = "hotel"
    target_radius = 5000
    target_type = 'lodging'

    params = {
        "location": target_coordinates,
        # "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    # convert response to json
    hotels_data = response.json()   
    # print(params)
    # print(hotels_data["results"][0]["name"])

    # print(hotels_data["results"][0])

    hotel_list.append(hotels_data["results"][0]["name"])
    
    
hotel_df["Hotel Name"] = hotel_list
hotel_df


C:\Users\rjtur\AppData\Local\Temp/ipykernel_9028/2032809379.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = hotel_list


,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind_Speed,Hotel Name
42,Luxor,25.6989,32.6421,21.96,23,0,2.32,Iberotel Luxor
94,San Isidro,-12.1167,-77.0500,21.12,75,0,3.09,Radisson Hotel Decapolis Miraflores
136,Bellview,30.4616,-87.3150,22.68,95,0,2.68,Hospitality Inn Management Co
250,Arvin,35.2091,-118.8284,21.01,51,0,1.91,Geo Rizo
304,Badiraguato,25.3667,-107.5167,22.61,37,0,1.30,Restaurante La palma
323,Marshall,34.3834,-86.3333,21.84,71,0,1.54,Riverview Campground
324,Ayia Anna,34.9399,33.4843,21.44,72,0,2.06,Anna Village Cyprus Bed and Breakfast
338,Meadville,41.6414,-80.1515,22.57,62,0,2.57,Holiday Inn Express Meadville (I-79 Exit 147A)...
613,Jandaia do Sul,-23.6031,-51.6433,22.36,89,0,3.24,Hotel Jandaia do Sul


In [ ]:
#---------------------------------------------
# STORE THE FIRST HOTEL RESULT INTO THE DATAFRAME
# PLOT MARKERS ON TOP OF THE HEATMAP
#---------------------------------------------

In [8]:

#  <dt>Country</dt><dd>{Country}</dd>


# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure


In [10]:
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Assign the marker layer to a variable
# markers = gmaps.marker_layer(locations)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig = gmaps.figure()
# fig = gmaps.figure()
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))